In [30]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [25]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [21]:
X = test[['sentence2_embedding_197',
 'sentence1_embedding_197', 
 'sentence2_embedding_287', 
 'sentence2_embedding_355', 
 'sentence1_embedding_287', 
 'sentence2_embedding_445',
 'sentence1_embedding_121', 
 'sentence1_embedding_355', 
 'sentence1_embedding_445', 'cosine_similarity']] 
X = X.astype(float)

y = test["similarity_score"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# create an xgboost regression model
model = XGBRegressor()

In [ ]:
#Hyper Parameter Tuned with Grid Search
param_grid = {"max_depth":    [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}


search = GridSearchCV(model, param_grid, cv=5).fit(X_train, y_train)
print("The best hyperparameters are ",search.best_params_)
#The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 500}

In [28]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

In [31]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_test, predictions) )
print("The score is %.5f" % RMSLE )

The score is 0.44124


In [32]:
from sklearn.metrics import mean_squared_error
ypred = regressor.predict(X_test)
mse = mean_squared_error(y_test, ypred)
print("MSE: %.2f" % mse)

print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 1.52
RMSE: 1.23


In [33]:
#WITHOUT Embeddings just with Cosine Similarity Score:
RMSLE = np.sqrt( mean_squared_log_error(y_test, X_test.cosine_similarity) )
print("The score is %.5f" % RMSLE )

The score is 0.70605


In [34]:
#WITHOUT Embeddings just with Cosine Similarity Score:
ypred = regressor.predict(X_test)
mse = mean_squared_error(y_test,  X_test.cosine_similarity)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 4.78
RMSE: 2.19
